In [1]:
import pandas as pd 
import os
import glob
import math
import datetime
import numpy as np
import pickle

# Get the names of subdirectories (citys from which data was pulled from NREL)
# in the "/Data" directory. Returns a list of these names.
def get_folder_names(data_folder_path):
    folder_names = [f for f in os.listdir(folder_path)\
        if not f.startswith('.')]
    return folder_names

# Turn a list of subdirectory names containing yearly csv data and a list of years into
# a dictionary of dictionaries. Ex: {city1 : {year1 : year1.csv, year2 : year2.csv}, city2 : {year1 : year1.csv, year2 : year2.csv}}
def data_dirs_2_csvdict(folder_path, folder_list, year_list):
    csv_dict = {}
    for folder in folder_list:
        year_dict = {}
        csv_list = [glob.glob(folder_path + '/{folder}/*_{year}.csv'.format(folder = folder, year = year))[0] for year in year_list]
        for year, csv in zip(year_list,csv_list):
            year_dict[year] = csv
        csv_dict[folder] = year_dict
    return csv_dict

# Turn a dictionary of dictionaries of csv files into a dictionary of dictionaries of pd
# dataframes
def dict_2_pd_dict(city_year_dict):
    df_dict = {}
    for city in city_year_dict:
        year_dict = {}
        for year, csv in city_year_dict[city].items():
            year_dict[year] = pd.read_csv(csv, header = 2)
        df_dict[city] = year_dict
    return df_dict

# Take in a dictionary of dictionaries of data frames, and return the same thing but 
# with only select columns from each. Rename the columns to include the city.
def select_cols(city_year_df_dict, col_names):
    df_dict = {}
    for city in city_year_df_dict:
        year_dict = {}
        for year, df in city_year_df_dict[city].items():
            year_dict[year] = df[col_names]
            new_cols = {}
            for col in col_names:
                new_cols[col] = '{city}_'.format(city=city) + col
            year_dict[year].rename(columns = new_cols, inplace = True)
        df_dict[city] = year_dict
    return df_dict

# Combine each year of data for a particular city into one data frame where the years
# are stacked on top of one another
def stack_years(city_year_df_dict):
    df_dict = {}
    for city in city_year_df_dict:
        df_list = []
        for df in city_year_df_dict[city].values():
            df_list.append(df)
        df_dict[city] = pd.concat(df_list, ignore_index=True)
    return df_dict

# Combine each city df horizontally to make one single df with the same number of rows, 
# but much "wider" now. 
def combine_cities(city_year_df_dict):
    full_df = pd.DataFrame()
    for df in city_year_df_dict.values():
        full_df = pd.concat([full_df, df], axis=1)
    return full_df

# Eliminate all but one column for those with the same data.
# Ex. Year, day, month...
# Inputs are a dataframe, a list of strings of col type names to replace 
# eg. ['Year', 'Hour'] and a string of the city to keep.
def one_col_keep(df, col_type_to_reduce, city_to_keep):
    cols = list(df.columns)
    cols_to_drop = []
    rename_dict = {}
    for col_type in col_type_to_reduce:
        drop_candidates = [col for col in cols if col_type in col]
        drop_list = [col for col in drop_candidates if city_to_keep not in col]
        rename_list = [col for col in drop_candidates if city_to_keep in col]
        if len(rename_list) == 1:
            rename_dict[rename_list[0]] = col_type
        elif len(rename_list) == 0:
            pass
        else:
            return Exception
        cols_to_drop.extend(drop_list)
    df = df.drop(cols_to_drop, axis=1)
    df = df.rename(columns = rename_dict)    
    return df


In [2]:
# Convert the month and day_of_month columns to a day_x and day_y
# column. Where both have values in range [-1,1] by moving cw around
# a unit circle where the top is Jan 1.
def normalize_day_of_year(df, month_col, day_col):
    month = df[month_col]
    month = month.to_numpy()
    day = df[day_col]
    day = day.to_numpy()
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    Day_X = np.zeros(len(month))
    Day_Y = np.zeros(len(month))
    for i in range(len(month)):
        day_of_year = sum(month_days[:month[i]-1]) + day[i]
        theta = 2 * math.pi * (day_of_year / 365)
        Day_X[i] = math.sin(theta)
        Day_Y[i] = math.cos(theta)
    days_x_y = pd.DataFrame({'Day_X':Day_X, 'Day_Y':Day_Y})
    df = pd.concat([days_x_y, df], axis=1)
    df = df.drop([month_col, day_col], axis=1)
    return df

# Convert the hour and minute columns to a time_x and time_y
# column. Where both have values in range [-1,1] by moving cw around
# a unit circle where the top is midnight.
def normalize_time_of_day(df, hour_col, minute_col):
    hour = df[hour_col]
    hour = hour.to_numpy()
    minute = df[minute_col]
    minute = minute.to_numpy()
    min_of_day = 60 * hour + minute
    Time_X = np.zeros(len(min_of_day))
    Time_Y = np.zeros(len(min_of_day))
    for i in range(len(min_of_day)):
        theta = 2 * math.pi * (min_of_day[i] / 1440)
        Time_X[i] = math.sin(theta)
        Time_Y[i] = math.cos(theta)
    time_x_y = pd.DataFrame({'Time_X':Time_X, 'Time_Y':Time_Y})
    df = pd.concat([time_x_y, df], axis = 1)
    df = df.drop([hour_col, minute_col], axis=1)
    return df
    
# Replaces all wind speed and wind direction columns with cartesian coordinates
# wind_x and wind_y
def wind_polar_2_cart(df, city_list):
    cols = list(df.columns)
    for city in city_list:
        w_speed_col = [col for col in cols if ((city in col) and ('Wind Speed' in col))]
        w_direction_col = [col for col in cols if ((city in col) and ('Wind Direction' in col))]
        if (len(w_speed_col) == 1 and len(w_direction_col)) == 1:
            pass
        else:
            
            return Exception
        w_speed_col = w_speed_col[0]
        w_direction_col = w_direction_col[0]
        w_speed = df[w_speed_col]
        w_direction = df[w_direction_col]
        w_speed = w_speed.to_numpy()
        w_direction = w_direction.to_numpy()
        Wind_X = np.zeros(len(w_speed))
        Wind_Y = np.zeros(len(w_speed))
        for i in range(len(w_speed)):
            w_direction_rad = math.radians(w_direction[i])
            Wind_X[i] = w_speed[i] * math.sin(w_direction_rad)
            Wind_Y[i] = w_speed[i] * math.cos(w_direction_rad)

        wind_x_label = '{city}_Wind_X'.format(city=city)
        wind_y_label = '{city}_Wind_Y'.format(city=city)
        Wind_X = pd.DataFrame({wind_x_label:Wind_X})
        Wind_Y = pd.DataFrame({wind_y_label:Wind_Y})

        df[w_speed_col]=Wind_X[wind_x_label]
        df[w_direction_col]=Wind_Y[wind_y_label]
        df = df.rename(columns = {w_speed_col : wind_x_label, w_direction_col : wind_y_label})  
    
    return df



In [6]:
# Local location of data files
path = os.getcwd() + '/Data'

# List of cities from which to get data (three letter abbreviation must match the name of the directory!)
full_city_list = ['BUT', 'CBG', 'ELV', 'GRE', 'JON', 'KIT', 'MGT', 'NCS', 'NPH', 'PIT', 'PKS', 'STU', 'WAS', 'WHE', 'YGT']
short_city_list = ['PIT', 'WAS', 'GRE', 'STU', 'ELV', 'KIT']
PIT_only = ['PIT']

'''
full_city_data_csv_dict = data_dirs_2_csvdict(path, full_city_list, list(range(1998, 2020)))
full_city_df_dict = dict_2_pd_dict(full_city_data_csv_dict)
full_city_dict = select_cols(GHI_df_dict_train_val, ['Clearsky GHI'])
GHI_dict = {}
for city in GHI_dict_train_val:
    df_list = []
    for df in GHI_dict_train_val[city].values():
        df_list.append(df)
    GHI_dict[city] = pd.concat(df_list,axis = 1 )
    GHI_dict[city][city + '_Clearsky_GHI_Max'] = GHI_dict[city].max(axis=1)
    GHI_dict[city] = GHI_dict[city].drop([city + '_Clearsky GHI'], axis=1)
GHI_train_dict = {}
for city in GHI_dict:
    GHI_train_dict[city] = pd.concat([GHI_dict[city], GHI_dict[city], GHI_dict[city]], ignore_index=True)

GHI_val_dict = GHI_dict
GHI_test_dict = GHI_dict
'''

city_lists = [full_city_list, short_city_list, PIT_only]

for i in range(len(city_lists)):
    
    # Get csv files in a dictionary for all desired data
    csv_dict = data_dirs_2_csvdict(path, city_lists[i], list(range(1998,2020)))

    # Convert csv files into Pandas dataframes in a dictionary
    df_dict = dict_2_pd_dict(csv_dict)

    # complete list of columns (if desired at some point):
    # #['Year','Month','Day','Hour','Minute','DHI','DNI','GHI','Clearsky DHI','Clearsky DNI','Clearsky GHI','Cloud Type', 'Dew Point', 'Solar Zenith', 'Surface Albedo', 'Wind Speed', 'Percipitable Water', 'Wind Direction', 'Relative Humidity', 'Temperature','Pressure','Global Horizontal UV Irradiance (280-400nm)', 'Global Horizontal UV Irradiance (295-385nm)']
    desired_features = ['Month','Day','Hour','Minute','DHI','DNI','GHI','Clearsky DHI','Clearsky DNI','Clearsky GHI', 'Dew Point', 'Surface Albedo', 'Wind Speed', 'Wind Direction', 'Relative Humidity', 'Temperature','Pressure','Global Horizontal UV Irradiance (280-400nm)', 'Global Horizontal UV Irradiance (295-385nm)']

    df_dict = select_cols(df_dict, desired_features)
    df_dict = stack_years(df_dict)

    df_full = combine_cities(df_dict)

    df_full = one_col_keep(df_full, ['Month','Day','Hour','Minute'], 'PIT')

    df_full = normalize_day_of_year(df_full, 'Month', 'Day')
    df_full = normalize_time_of_day(df_full,'Hour', 'Minute')

    df_full = wind_polar_2_cart(df_full, city_lists[i])
        
        
    if i == 0:
        df_full_cities = df_full

    elif i == 1:
        df_short_cities = df_full
        
    elif i == 2:
        df_PIT = df_full

full_cities_dict = {
    'complete_data' : df_full_cities
    }
short_cities_dict = {
    'complete_data' : df_short_cities
}
PIT_dict = {
    'complete_data' : df_PIT
    }

with open('full_cities_dict.pkl', 'wb') as f:
    pickle.dump(full_cities_dict, f)

with open('short_cities_dict.pkl', 'wb') as f:
    pickle.dump(short_cities_dict, f)
    
with open('PIT_dict.pkl', 'wb') as f:
    pickle.dump(PIT_dict, f)
